In [6]:
!pip install tabulate
from dataclasses import dataclass
from bs4 import BeautifulSoup
from tabulate import tabulate
import requests
def get_price_by_usd(currency):
    url = f"https://www.google.com/finance/quote/{currency}-USD"
    r =requests.get(url)
    soup = BeautifulSoup(r.content, "html.parser")
    price_fx = soup.find('div', attrs= {"data-last-price":True})
    fx = float(price_fx['data-last-price'])
    return fx

    
    
def get_price_info(ticker,exchange):
    url = f"https://www.google.com/finance/quote/{ticker}:{exchange}"
    r = requests.get(url)
    soup = BeautifulSoup(r.content, "html.parser")
    price_div = soup.find('div', attrs= {"data-last-price":True})
    price = float(price_div["data-last-price"])
    currency = price_div["data-currency-code"]

    usd_price = price
    if currency != 'USD':
        fx = get_price_by_usd(currency)
        usd_price = round(price*fx,2)
        
    return{
        "ticker":ticker,
        "exchange":exchange,
        "price":price,
        "currency":currency,
        "usd_price":usd_price
    }

def display_portfolio_summary(portfolio):
    if not isinstance(portfolio, Portfolio):
        raise TypeError("Please provide an instance of the Portfolio type")
    portfolio_value = portfolio.get_total_value()

    position_data = []

    for position in portfolio.positions:
        position_data.append([
            position.stock.ticker,
            position.stock.exchange,
            position.quantity,
            position.stock.usd_price,
            position.quantity * position.stock.usd_price,
            position.quantity * position.stock.usd_price / portfolio_value *100
        ])
    print(tabulate(position_data,
                      headers = ["Ticker", "Exchange", "Quantity", "Price",
                                "Market Value", "% Allocation"],
                      tablefmt = 'psql',
                      floatfmt = ".2f"))
    print(f"Total portfolio value: {portfolio_value: ,.2f}.")

@dataclass
class Stock:
    ticker: str
    exchange: str
    currency: str = 'USD'
    price: float =0
    usd_price: float = 0
    def __post_init__(self):
        price_info = get_price_info(self.ticker,self.exchange)
        if price_info['ticker'] == self.ticker:
            self.price = price_info['price']
            self.currency = price_info['currency']
            self.usd_price = price_info['usd_price']

@dataclass
class Position:
    stock:Stock
    quantity: int

@dataclass
class Portfolio:
    positions: list[Position]
    
    def get_total_value(self):
        total_value = 0
        for position in self.positions:
            total_value += position.quantity * position.stock.usd_price
            
        return total_value
        


        

        



if __name__== "__main__":
    # print(get_price_info('SHOP',"TSE"))
    # print(get_price_by_usd('CAD'))
    shop = Stock('SHOP','TSE')
    msft = Stock('MSFT','NASDAQ')
    google = Stock('GOOGL','NASDAQ')
    portfolio = Portfolio([Position(shop,10), Position(msft,50), Position(google,10)])
    # print(round(portfolio.get_total_value(),2))

display_portfolio_summary(portfolio)
    




    


    



+----------+------------+------------+---------+----------------+----------------+
| Ticker   | Exchange   |   Quantity |   Price |   Market Value |   % Allocation |
|----------+------------+------------+---------+----------------+----------------|
| SHOP     | TSE        |         10 |   67.58 |         675.80 |           2.75 |
| MSFT     | NASDAQ     |         50 |  442.57 |       22128.50 |          90.06 |
| GOOGL    | NASDAQ     |         10 |  176.79 |        1767.90 |           7.19 |
+----------+------------+------------+---------+----------------+----------------+
Total portfolio value:  24,572.20.
